In [1]:
from pyiceberg.catalog import load_catalog
from pyiceberg.exceptions import TableAlreadyExistsError
from pyiceberg.schema import Schema
from pyiceberg.types import StringType, DoubleType, NestedField
import pandas as pd
from IPython.display import display
import pyarrow as pa
from minio import Minio

In [3]:
MINIO_URL = "http://172.18.0.9:9000"
MINIO_ENDPOINT = "172.18.0.9:9000"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin"

########################### REST Catalog  ###########################
RAW_CATALOG_URL = "http://172.18.0.10"
RAW_CATALOG_PORT = "8181"
RAW_CATALOG_NAME = "RAW"

TABLE_CATALOG_URL = "http://172.18.0.5"
TABLE_CATALOG_PORT = "8182"
TABLE_CATALOG_NAME = "TABLE"

# Charger le catalogue Iceberg RAW
catalogRAW = load_catalog('RAW', **{
    'uri': f"{RAW_CATALOG_URL}:{RAW_CATALOG_PORT}",
    's3.endpoint': MINIO_URL,
    's3.access-key-id': MINIO_ACCESS_KEY,
    's3.secret-access-key': MINIO_SECRET_KEY,
})

In [4]:
minio_client = Minio(
    MINIO_ENDPOINT,
    access_key=MINIO_ACCESS_KEY,
    secret_key=MINIO_SECRET_KEY,
    secure=False
)

In [6]:
for obj in minio_client.list_objects("landing"):
    print(obj.object_name)

Design sans titre.png
employees.csv
new/
test/


In [8]:
data = minio_client.get_object(
        bucket_name="landing",
        object_name="employees.csv",
    )

In [9]:
data = pd.read_csv(data)

In [10]:
data.head()

,EmployeeID,FirstName,LastName,Department,Position,Email
0,101,John,Doe,Engineering,Software Engineer,john.doe@example.com
1,102,Jane,Smith,Marketing,Marketing Manager,jane.smith@example.com
2,103,Emily,Davis,Sales,Sales Representative,emily.davis@example.com
3,104,Michael,Brown,Human Resources,HR Specialist,michael.brown@example.com
4,105,Sarah,Wilson,Finance,Accountant,sarah.wilson@example.com


In [3]:
schema = Schema(
    NestedField(1, "city", StringType(), required=False),
    NestedField(2, "lat", DoubleType(), required=False),
    NestedField(3, "long", DoubleType(), required=False),
)

table_name = "my_namespace.cities_1"  # Specify your table name

try: 
    # Create the table
    my_table = catalogRAW.create_table(
        identifier=table_name,
        schema=schema,
    )
except TableAlreadyExistsError:
    print(f"Table {table_name} already exists in the catalog")

Table my_namespace.cities_1 already exists in the catalog


In [ ]:
df = pa.Table.from_pylist(
    [
        {"city": "Amsterdam", "lat": 52.371807, "long": 4.896029},
        {"city": "San Francisco", "lat": 37.773972, "long": -122.431297},
        {"city": "Drachten", "lat": 53.11254, "long": 6.0989},
        {"city": "Paris", "lat": 48.864716, "long": 2.349014},
    ],
)

In [ ]:
table = catalogRAW.load_table("my_namespace.cities_1")
table.overwrite(df)

In [4]:
scan = catalogRAW.load_table("my_namespace.cities_1").scan()
pandas_df = scan.to_pandas()
df = pd.DataFrame(pandas_df)
display(df)

,city,lat,long
0,Amsterdam,52.371807,4.896029
1,San Francisco,37.773972,-122.431297
2,Drachten,53.112540,6.098900
3,Paris,48.864716,2.349014
